Introduction to Natural Language Processing
* Getting the necessary doc and files

In [ ]:
# !wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
# !wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/refs/heads/main/extras/helper_functions.py

--2024-12-11 20:30:43--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 216.58.212.123, 142.250.200.251, 172.217.18.59, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|216.58.212.123|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: 'nlp_getting_started.zip'

     0K .......... .......... .......... .......... ..........  8%  145K 4s
    50K .......... .......... .......... .......... .......... 16%  280K 3s
   100K .......... .......... .......... .......... .......... 25%  531K 2s
   150K .......... .......... .......... .......... .......... 33%  559K 1s
   200K .......... .......... .......... .......... .......... 42%  721K 1s
   250K .......... .......... .......... .......... .......... 50%  667K 1s
   300K .......... .......... .......... .......... .......... 59%  381K 1s
   350K .......... .......... ..........

In [3]:
from helper_functions import unzip_data, plot_loss_curves, create_tensorboard_callback, compare_historys
unzip_data('nlp_getting_started.zip')

In [1]:
import pandas as pd
import numpy as np

train_data = pd.read_csv(r'train.csv')
test_data = pd.read_csv(r'test.csv')
train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [2]:
# shuffling the train_data
train_data = train_data.sample(frac=1, random_state=42)
train_data.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [3]:
train_data.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [4]:
len(train_data), len(test_data)

(7613, 3263)

In [5]:
import random
random_index = random.randint(0, len(train_data)-5)

for row in train_data[['text', 'target']][random_index:random_index+5].itertuples():
    _, text, target = row
    print(f"Target :{target} ", "(real diaster)" if target>0 else "(not real diseater)")
    print(f"Text :{text} \n")
    print('......\n')

Target :0  (not real diseater)
Text :GREAT PERFORMANCE CHIP FUEL/GAS SAVER CHEVY TAHOE/BLAZER/AVALANCHE/S-10 http://t.co/iCrZi5TqC5 http://t.co/ONxhKfHn2a 

......

Target :1  (real diaster)
Text :$1 million bail for man accused of #shooting at Fife #police - Aug 5 @ 8:16 PM ET http://t.co/hu5CXqnoBf 

......

Target :1  (real diaster)
Text :VIDEO: 'We're picking up bodies from water': Rescuers are searching for hundreds of migrants in the... http://t.co/UymxocFs33 #BBC #News 

......

Target :0  (not real diseater)
Text :@lauren_miller_7 she won't harm you 

......

Target :1  (real diaster)
Text :I was in a horrible car accident this past Sunday. I'm finally able to get around. Thank you GOD?? 

......



Spliting data into training, testing anad validation sets

In [6]:
from sklearn.model_selection import train_test_split
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_data['text'].to_numpy(), 
                                                                            train_data['target'].to_numpy(),
                                                                            test_size=0.1, 
                                                                            random_state=42) 

In [7]:
len(train_sentences), len(val_sentences)

(6851, 762)

In [8]:
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

**converting text to numbers**
* Tokenization: direct mapping of tekens(a token could be a word or a character)
* Embedding: creating a metrix of feature vectors for each token(s)(the size of the feature vector can be define and the embedding can be learned)

In [9]:
# Tokenization
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

In [11]:
text_vectorizer = TextVectorization(max_tokens=None,
                                    standardize = 'lower_and_strip_punctuation',
                                    split = 'whitespace',
                                    ngrams= None,
                                    output_mode = 'int',
                                    output_sequence_length = None,
                                    pad_to_max_tokens = False)

In [24]:
len(train_sentences[0].split())

7

In [23]:
# findinf the number of avg word in ine training sentences
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [25]:
# setting up text vectorizer
max_vocab_length = 10000
max_length = 15
text_vectorizer = TextVectorization(
    max_tokens = max_vocab_length,
    output_mode = 'int',
    output_sequence_length = max_length
)

In [26]:
text_vectorizer.adapt(train_sentences)

In [27]:
# creating a smaple sentence tokens

sentence = "This is a sample sentence"
text_vectorizer([sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  19,    9,    3, 8839,    1,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0]], dtype=int64)>

In [33]:
# choosing the random sentence in the training set
random_sentence = random.choice(train_sentences)
print(f"Origin text: \n{random_sentence}\n")

text_vectorizer([random_sentence])

Origin text: 
'Mages of Fairy Tail.. Specialize in property damage!' - Natsu Dragneel



<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   1,    6, 5672, 7826, 8242,    4,  927,  233,    1,    1,    0,
           0,    0,    0,    0]], dtype=int64)>